Se importan las librerias y paquetes requidos

In [567]:
import pandas as pd
import numpy as np 
import json
import ast
from sklearn.preprocessing import LabelEncoder

In [568]:
df = pd.read_csv('./origen.csv')
test = pd.read_csv('./testear.csv')

# Limpieza de datos
## Variables de tipo numericas
- average rating
- numVotes
- isAdult
- startYear
- endYear
- runtimeMinutes
- seasonNumber
- episodeNumber
- ordering
- language
- isOriginalTitle
- budget
- popularity
- revenue
- runtime

## Variables de tipo str
- titleType
- genres_x
- directors
- writers
- adult
- genres_y
- original_language
- production_company
- production_country
- status
- tagline
- video

## Variables booleanas

Todas las columnas son rotuladas de manera numerica
Las columnas indice, directors, writer, tagline, language, adult fueron eliminadas

In [569]:
df = df.drop('Unnamed: 0', axis=1)

In [570]:
print(df['genres_x'].str.count(',').max())
# Se pueden tener como maximo 3 generos x

df[['generox1', 'generox2', 'generox3']] = df['genres_x'].str.split(',', expand=True)
df = df.drop('genres_x', axis = 1)

2.0


In [571]:
generox = LabelEncoder()
df["generox1_encoded"] = generox.fit_transform(df["generox1"])
df["generox2_encoded"] = generox.fit_transform(df["generox2"])
df["generox3_encoded"] = generox.fit_transform(df["generox3"])
df['generox1'] = df['generox1_encoded']
df['generox2'] = df['generox2_encoded']
df['generox3'] = df['generox3_encoded']
df = df.drop('generox1_encoded', axis = 1)
df = df.drop('generox2_encoded', axis = 1)
df = df.drop('generox3_encoded', axis = 1)

firstValueGenerox1 = df['generox1'].iloc[50]  
df['generox1'] = df['generox1'].apply(lambda x: None if x == firstValueGenerox1 else x)

firstValueGenerox2 = df['generox2'].iloc[3]  
df['generox2'] = df['generox2'].apply(lambda x: None if x == firstValueGenerox2 else x)

firstValueGenerox2 = df['generox3'].iloc[3]  
df['generox3'] = df['generox3'].apply(lambda x: None if x == firstValueGenerox2 else x)

In [572]:
leAdult = LabelEncoder()
df["isAdult_encoded"] = leAdult.fit_transform(df["isAdult"])
df['isAdult'] = df['isAdult_encoded']
df = df.drop('isAdult_encoded', axis = 1)

df = df.drop('adult', axis = 1)

In [573]:
leTitleType = LabelEncoder()
df["titleType_encoded"] = leTitleType.fit_transform(df["titleType"])
df['titleType'] = df['titleType_encoded']
df = df.drop('titleType_encoded', axis = 1)

In [593]:
leLanguage = LabelEncoder()
df["language_encoded"] = leLanguage.fit_transform(df["original_language"])
df['original_language'] = df['language_encoded']
df = df.drop('language_encoded', axis = 1)

firstValueLanguage = df['original_language'].iloc[0]  
df['original_language'] = df['original_language'].apply(lambda x: None if x == firstValueLanguage else x)

df = df.drop('language', axis = 1)

In [575]:
leStatus = LabelEncoder()
df["status_encoded"] = leLanguage.fit_transform(df["status"])
df['status'] = df['status_encoded']
df = df.drop('status_encoded', axis = 1)

firstValueStatus = df['status'].iloc[0]  
df['status'] = df['status'].apply(lambda x: None if x == firstValueStatus else x)

In [576]:
leVideo = LabelEncoder()
df["video_encoded"] = leAdult.fit_transform(df["video"])
df['video'] = df['video_encoded']
df = df.drop('video_encoded', axis = 1)

firstValueVideo = df['video'].iloc[0]  
df['video'] = df['video'].apply(lambda x: None if x == firstValueVideo else x)

In [577]:
leAttributes = LabelEncoder()
df["attributes_encoded"] = leAdult.fit_transform(df["attributes"])
df['attributes'] = df['attributes_encoded']
df = df.drop('attributes_encoded', axis = 1)

firstValueAtrubutes = df['attributes'].iloc[0] 
df['attributes'] = df['attributes'].apply(lambda x: None if x == firstValueAtrubutes else x)

In [578]:
print(df['writers'].str.count(',').max())
# Se pueden tener hasta 1288 escritores

print(df['directors'].str.count(',').max())
# Se pueden tener hasta 475 directores

df = df.drop('writers', axis = 1)
df = df.drop('directors', axis = 1)

1287
475


In [579]:
df = df.drop('tagline', axis = 1)

In [580]:
def extract_name(row):
    if isinstance(row, list) and len(row) > 0:
        company_dict = row[0]
        if 'name' in company_dict:
            return company_dict['name']
    return ""

In [581]:
def extract_and_join_names(row):
    if pd.notna(row):  
        cleaned_json = row.replace("'", '"').strip() 
        try:
            country_list = json.loads(cleaned_json)
            names = [country['name'] for country in country_list]
            return ', '.join(names)
        except json.JSONDecodeError as e:
            return 'Invalid JSON'
    else:
        return ''

In [582]:
df['countries'] = df['production_countries'].apply(extract_and_join_names)

In [583]:
df['companies'] = df['production_companies'].apply(extract_and_join_names)

In [584]:
df['genre_y'] = df['genres_y'].apply(extract_and_join_names)

In [585]:
def extract_column_list(column, key):
    keys = []

    for i in range(df[column].str.count(', ').max() + 1):
        keys.append(key.format(n=i))

    df[keys] = df[column].str.split(',', expand=True)

In [586]:
extract_column_list('genre_y', 'genre_y_{n:d}')
df.drop('genre_y', axis=1, inplace=True)
df.drop('genres_y', axis=1, inplace=True)

In [587]:
extract_column_list('companies', 'companies_{n:d}')
df.drop('companies', axis=1, inplace=True)
df.drop('production_companies', axis=1, inplace=True)

In [588]:
extract_column_list('countries', 'countries_{n:d}')
df.drop('countries', axis=1, inplace=True)
df.drop('production_countries', axis=1, inplace=True)

In [589]:
object_columns = df.select_dtypes(include=['object']).columns
print(object_columns)

Index(['original_language', 'status', 'video', 'genre_y_0', 'genre_y_1',
       'genre_y_2', 'genre_y_3', 'genre_y_4', 'genre_y_5', 'genre_y_6',
       'genre_y_7', 'companies_0', 'companies_1', 'companies_2', 'companies_3',
       'companies_4', 'companies_5', 'companies_6', 'companies_7',
       'companies_8', 'companies_9', 'companies_10', 'companies_11',
       'companies_12', 'companies_13', 'companies_14', 'companies_15',
       'companies_16', 'companies_17', 'companies_18', 'companies_19',
       'companies_20', 'companies_21', 'companies_22', 'companies_23',
       'companies_24', 'companies_25', 'countries_0', 'countries_1',
       'countries_2', 'countries_3', 'countries_4', 'countries_5',
       'countries_6', 'countries_7', 'countries_8', 'countries_9',
       'countries_10', 'countries_11', 'countries_12', 'countries_13',
       'countries_14', 'countries_15', 'countries_16', 'countries_17',
       'countries_18', 'countries_19', 'countries_20', 'countries_21',
       'co

In [590]:
keyGenre = 'genre_y_{n:d}'
key_encodedGenre = 'genre_y_{n:d}_encoded'

leGenre = LabelEncoder()
for n in range(8):  
    column_name = f'genre_y_{n}'
    
    if column_name in df:
        encoded_genrey = f'{column_name}_encoded'
        df[encoded_genrey] = leGenre.fit_transform(df[column_name])
        df[column_name] = df[encoded_genrey]
        df.drop(encoded_genrey, axis=1, inplace=True)

        firstValue = df[column_name].iloc[0]  # Obtiene el primer valor de la columna
        df[column_name] = df[column_name].apply(lambda x: None if x == firstValue else x)

In [591]:
keyCountries = 'countries_{n:d}'
key_encodedCountries = 'countries_{n:d}_encoded'

leCountries = LabelEncoder()
for n in range(24):  
    column_name = f'countries_{n}'
    
    if column_name in df:
        encoded_countries = f'{column_name}_encoded'
        df[encoded_countries] = leCountries.fit_transform(df[column_name])
        df[column_name] = df[encoded_countries]
        df.drop(encoded_countries, axis=1, inplace=True)

        firstValue = df[column_name].iloc[0]  # Obtiene el primer valor de la columna
        df[column_name] = df[column_name].apply(lambda x: None if x == firstValue else x)

In [592]:
keyCompanies = 'companies_{n:d}'
key_encodedCompanies = 'companies_{n:d}_encoded'

leCompanies = LabelEncoder()
for n in range(25):  
    column_name = f'companies_{n}'
    
    if column_name in df:
        encoded_companies = f'{column_name}_encoded'
        df[encoded_companies] = leCompanies.fit_transform(df[column_name])
        df[column_name] = df[encoded_companies]
        df.drop(encoded_companies, axis=1, inplace=True)

print(df['companies_0'])

0            0
1            0
2            0
3            0
4            0
          ... 
977536       0
977537    8088
977538       0
977539       0
977540       0
Name: companies_0, Length: 977541, dtype: int64


**Nota** : Se ha decidido dejar los valores nulos en el dataset como tales, en lugar de reemplazarlos por un numero. Por esta razón, se utilizaran modelos que admitan este tipo de dato.

In [595]:
print(df.columns)

Index(['averageRating', 'numVotes', 'titleType', 'isAdult', 'startYear',
       'endYear', 'runtimeMinutes', 'seasonNumber', 'episodeNumber',
       'ordering', 'attributes', 'isOriginalTitle', 'budget',
       'original_language', 'popularity', 'revenue', 'runtime', 'status',
       'video', 'generox1', 'generox2', 'generox3', 'genre_y_0', 'genre_y_1',
       'genre_y_2', 'genre_y_3', 'genre_y_4', 'genre_y_5', 'genre_y_6',
       'genre_y_7', 'companies_0', 'companies_1', 'companies_2', 'companies_3',
       'companies_4', 'companies_5', 'companies_6', 'companies_7',
       'companies_8', 'companies_9', 'companies_10', 'companies_11',
       'companies_12', 'companies_13', 'companies_14', 'companies_15',
       'companies_16', 'companies_17', 'companies_18', 'companies_19',
       'companies_20', 'companies_21', 'companies_22', 'companies_23',
       'companies_24', 'companies_25', 'countries_0', 'countries_1',
       'countries_2', 'countries_3', 'countries_4', 'countries_5',
       

# Modelo Baseline

In [ ]:
data = df.fillna(df.mean())

